In [3]:
# pip install transformers
# pip install torch
# pip install python-dotenv 
import os
import dotenv
import joblib
from huggingface_hub import hf_hub_download
import torch 

dotenv.load_dotenv()
HUGGING_FACE_API_KEY = os.getenv("HUGGING_FACE_API_KEY")

In [6]:
# model_id = "mosama/Llama-2-Medical-Merged-LoRA" 
# filenames = ["config.json","generation_config.json", "model-00004-of-00004.safetensors", "model.safetensors.index.json"]

# "pytorch_model.bin", "special_tokens_map.json", "tokenizer_config.json"]
# "added_tokens.json",  "generation_config.json","spiece.model"

model_id = "medicalai/ClinicalBERT"
filenames = ["config.json","pytorch_model.bin", "special_tokens_map.json",  "tokenizer_config.json"]
for filename in filenames:
        downloaded_model_path = hf_hub_download(
                repo_id=model_id,
                filename=filename,
                token=HUGGING_FACE_API_KEY
                )
print(downloaded_model_path)

/Users/jsauza/.cache/huggingface/hub/models--medicalai--ClinicalBERT/snapshots/44dfd560680cb779c0c7d473c96d098d778a2654/tokenizer_config.json


### Fill Mask Model

In [8]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = "medicalai/ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)
model_checkpoint = model_id #"distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
pipeline = pipeline("fill-mask", model=model, device=-1, tokenizer=tokenizer)

In [9]:
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> ClinicalBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> ClinicalBERT number of parameters: 135M'
'>>> BERT number of parameters: 110M'


In [10]:
text = "The patient has a high fever which was indicative of [MASK]."

In [11]:
inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> The patient has a high fever which was indicative of infection.'
'>>> The patient has a high fever which was indicative of pain.'
'>>> The patient has a high fever which was indicative of symptoms.'
'>>> The patient has a high fever which was indicative of fever.'
'>>> The patient has a high fever which was indicative of influenza.'


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipeline = pipeline("text2text-generation", model=model, device=-1, tokenizer=tokenizer, max_length=1000)

Loading checkpoint shards: 100%|██████████| 2/2 [00:25<00:00, 12.80s/it]


In [13]:
pipeline("What do you build a simple electronic engine?")
#pipeline("What are competitors to Apache Kafka?")

[{'generated_text': 'You can build a simple electronic engine by using a battery and a circuit board.'}]

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
['Pour a cup of bolognese into a large bowl and add the pasta']

/Users/jsauza/Documents/GitHub/Med_LLM/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Pour a cup of bolognese into a large bowl and add the pasta']


['Pour a cup of bolognese into a large bowl and add the pasta']

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
# model = AutoModelForSeq2SeqLM.from_pretrained("WizardLM/WizardCoder-15B-V1.0")
model = AutoModelForSeq2SeqLM.from_pretrained("WizardLM/WizardCoder-Python-34B-V1.0")

ValueError: Unrecognized configuration class <class 'transformers.models.llama.configuration_llama.LlamaConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("WizardLM/WizardCoder-15B-V1.0")
model = AutoModelForCausalLM.from_pretrained("WizardLM/WizardCoder-15B-V1.0")

: 

: 

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel

# model_name = "WizardLM/WizardCoder-Python-34B-V1.0"
model_name = "WizardLM/WizardCoder-3B-V1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

inputs = tokenizer("How do you make a lemonade?", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

TypeError: The current model class (GPTBigCodeModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'GPTBigCodeForCausalLM'}

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("WizardLM/WizardCoder-15B-V1.0")
tokenizer = AutoTokenizer.from_pretrained("WizardLM/WizardCoder-15B-V1.0")

inputs = tokenizer("How do you make a lemonade?", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
#['Pour a cup of bolognese into a large bowl and add the pasta']

ValueError: Unrecognized configuration class <class 'transformers.models.gpt_bigcode.configuration_gpt_bigcode.GPTBigCodeConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [ ]:
inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))